In [ ]:
import math
import numpy as np

In [ ]:
# Initialize empty load/moment arrays. These act as defaults—
# specific loads override the empty arrays if provided.
pointLoads = np.array([[]]) # [location,xMag, yMag]
pointMoments = np.array([[]]) # [location, Mag]
distributedLoads = np.array([[]]) # [location i, location f, Mag]
linearLoads = np.array([[]]) # [location i, location f, startMag, endMag]

In [ ]:

#inputs

def get_positive_input(prompt):
    while True:
        user_input = input(prompt).strip()
        if user_input == "":
            print(" Input cannot be empty. Try again.")
            continue
        try:
            val = float(user_input)
            if val < 0:
                print(" Value cannot be negative. Try again.")
                continue
            return val
        except ValueError:
            print(" Invalid number. Please enter a valid numeric value.")

# User input for beam parameters
span = get_positive_input("Enter the span of the beam in meters: ")
A = get_positive_input("Enter the location of support A (smooth pin / hinge) in meters: ")
B = get_positive_input("Enter the location of support B (roller/rocker) in meters: ")

# Optional: Print confirmation
print(f"\nBeam span = {span} m")
print(f"Support A at = {A} m")
print(f"Support B at = {B} m")



#force data

import numpy as np

def parse_input(prompt, dims):
    """
    Helper to take multiline input for loads.
    dims = number of elements per row (e.g. 3 for [location, xMag, yMag])
    """
    print(prompt)
    print(f"Enter values as space-separated per line ({dims} values per row), or leave empty and press Enter to skip.")
    print("Example: for {dims} values: {' '.join(['val'+str(i+1) for i in range(dims)])}")
    print("Type 'done' when finished.\n")

    rows = []
    while True:
        line = input("> ").strip()
        if line.lower() == "done":
            break
        if not line:
            continue
        try:
            values = [float(x) for x in line.split()]
            if len(values) != dims:
                print(f"⚠️ Please enter exactly {dims} values.")
                continue
            rows.append(values)
        except ValueError:
            print("⚠️ Invalid number format. Try again.")

    if not rows:
        return np.zeros((1, dims))  # <-- THIS is the safe placeholder
    else:
        return np.array(rows)


# USER INPUT SECTION
pointLoads = parse_input("Enter Point Loads [location, xMag, yMag]:", 3)
pointMoments = parse_input("Enter Point Moments [location, Mag]:", 2)
distributedLoads = parse_input("Enter Distributed Loads [location i, location f, Mag]:", 3)
linearLoads = parse_input("Enter Linear Loads [location i, location f, startMag, endMag]:", 4)

print("\n--- Summary of Entered Loads ---")
print("Point Loads:\n", pointLoads)
print("Point Moments:\n", pointMoments)
print("Distributed Loads:\n", distributedLoads)
print("Linear Loads:\n", linearLoads)



In [ ]:
#Defaults

divs = 10000 #Divide the span up into this number of data points
delta = span/divs # Distance between data points
X = np.arange(0, span+delta, delta) # range of X-coordinates
nPL = len(pointLoads[0]) # test for pointLoads
nPM = len(pointMoments[0]) #test for pointMoments 
nUDL = len(distributedLoads[0]) #test for UDL (Uniformly Distributed Loads) 
nLDL = len(linearLoads[0]) #test for LDL (Linearly Distributed Loads)


reactions = np.array([0.0, 0 , 0]) # for (Va, Ha, Vb)
shearForce = np.empty([0,len(X)]) # Shear forces at each data point
bendingMoment = np.empty([0,len(X)]) #Bending moment at each data point

Function to calculate reactions due to point loads

In [ ]:
def reactions_PL(n):
  xp = pointLoads[n, 0] # location of point load
  fx =pointLoads[n, 1]  # point load horizontal component Mag
  fy =pointLoads[n, 2]   # point load vertical component Mag

  la_p = A - xp #Lever arm of point load about point A
  mp = fy*la_p  #Moment generated by point load about A ( CW moments are positive )
  la_vb = B-A #Lever arm of evrtical reaction at B about point A 

  Vb = mp/la_vb #Vertical reaction at B
  Va = -fy-Vb #Vertical reaction at A
  Ha = -fx  #Horizontal reaction at A

  return Va, Vb, Ha


Function to calculate reactions due to point Moments

In [ ]:
def reactions_PM(n):
    xm = pointMoments[n,0] #Location of point moment
    m = pointMoments[n,1] #Point moment magnitude
    la_vb = B-A #Lever arm of vertical reaction at B about point A

    Vb = m/la_vb #Vertical reaction at B
    Va = -Vb #Vertical reaction at A

    return Va, Vb

Function to calculate reactions due to UDLS

In [ ]:
def reactions_UDL(n):
    xStart = distributedLoads[n,0]
    xEnd = distributedLoads[n,1]
    fy = distributedLoads[n,2]

    fy_Res = fy*(xEnd-xStart)
    x_Res = xStart + 0.5*(xEnd-xStart)

    la_p = A - x_Res #Lever arm of resultant point load about point A
    mp = fy_Res*la_p #Moment generated by resultant point load about A (CW moments are positive)
    la_vb = B-A #Lever arm of vertical reaction at B about point A

    Vb = mp/la_vb  #Vertical reaction at B
    Va = -fy_Res-Vb  #Vertical reaction at A

    return Va, Vb


    


Function to calculate reactions due to LDLs

In [ ]:
def reactions_LDL(n):
    xStart = linearLoads[n,0]
    xEnd = linearLoads[n,1]
    fy_start = linearLoads[n,2]
    fy_end = linearLoads[n,3]
    
 #Determine the location and the magnitude of force resultant
    if abs(fy_start)>0:
        fy_Res = 0.5*fy_start*(xEnd-xStart)
        x_Res = xStart + (1/3)*(xEnd-xStart)
    else:
        fy_Res = 0.5*fy_end*(xEnd-xStart)
        x_Res = xStart + (2/3)*(xEnd-xStart)

    la_p = A - x_Res  #Lever arm of resultant point laod about point A
    mp = fy_Res*la_p  #Moment generated by resultant point load about A (CW moments are positive)
    la_vb = B-A  #Lever arm of vertical reaction at B aboud point A

    Vb = mp/la_vb  #Vertical reaction at B
    Va = -fy_Res-Vb  #Vertical reaction at A

    return Va, Vb

    

Cycle through all point loads and determine reactions

In [ ]:
PL_record = np.empty([0,3])
if (nPL>0):

    for n, p in enumerate(pointLoads):
      va, vb, ha = reactions_PL(n) #Calculate reactions
      PL_record = np.append(PL_record, [np.array([va, ha, vb])], axis=0) #Storing reactions for each point load

      #Add reactions to record
      reactions[0] = reactions[0] + va
      reactions[1] = reactions[1] + ha
      reactions[2] = reactions[2] + vb


Cycle through all point moments and determine reactions

In [ ]:
PM_record = np.empty([0,2])
if (nPM>0):

    for n, p in enumerate(pointMoments):
        va, vb = reactions_PM(n)
        PM_record = np.append(PM_record, [np.array([va, vb])], axis=0) #Storing reactions for each point moment

        #Add reactions to record
        reactions[0] = reactions[0] + va
        reactions[2] = reactions[2] + vb



Cycle through all UDLs and determine reactions

In [ ]:
UDL_record = np.empty([0,2])
if (nUDL>0):

    for n, p in enumerate(distributedLoads):
        va, vb = reactions_UDL(n)
        UDL_record = np.append(UDL_record, [np.array([va, vb])], axis=0) #Storing reactions for each UDL
        #Add reactions to record
        reactions[0] = reactions[0] + va
        reactions[2] = reactions[2] + vb

Cycle through all LDLs and determine reactions

In [ ]:
LDL_record = np.empty([0,2])
if (nLDL>0):

    for n, p in enumerate(linearLoads):
        va, vb = reactions_LDL(n)
        LDL_record = np.append(LDL_record, [np.array([va, vb])], axis=0) #Storing reactions for each LDL
        #Add reactions to record
        reactions[0] = reactions[0] + va
        reactions[2] = reactions[2] + vb

Cycle through all point loads and determine shear and moment

In [ ]:
def shear_moment_PL(n):
  xp = pointLoads[n, 0] # location of point load
  fy =pointLoads[n, 2]   # Point load vertical componen Mag
  Va = PL_record[n,0]  # Vertical reaction at A for this point load
  Vb = PL_record[n,2]  # Vertical reaction at B for this point load

    # Cycle through the structure and calculate the shear force and bending moment at each point
  Shear = np.zeros(len(X))  #Initialise a container to hold all shear force data for this point load
  Moment = np.zeros(len(X)) #Initialise a container to hold all shear force data for this point load

  for i, x in enumerate(X):
    shear = 0  #Initialise the shear force for this data point
    moment = 0  #Initialise the bending moment for this data point

    if x>A:        # calculate the shear and moment due to reaction at A
      shear = shear + Va
      moment = moment - Va*(x-A)

    if x>B:  # calculate the shear and moment due to reaction at B
      shear = shear + Vb
      moment = moment - Vb*(x-B)

    if x>xp:   # calculate the shear and moment due to point load
      shear = shear + fy
      moment = moment - fy*(x-xp)
        
    # Store shear and moment for this location      
    Shear[i] = shear
    Moment[i] = moment

  return Shear, Moment



Define function to calculate shear forces and bending moments due to point moments

In [ ]:
def shear_moment_PM(n):
    xm = pointMoments[n,0] #location of point moment
    m = pointMoments[n,1] # point moment magnitude
    Va = PM_record[n,0] # vertical reaction at A for this point moment
    Vb = PM_record[n,1]  # # vertical reaction at B for this point moment

     # Cycle through the structure and calculate the shear force and bending moment at each point
    Shear = np.zeros(len(X))  #Initialise a container to hold all shear force data for this point load
    Moment = np.zeros(len(X)) #Initialise a container to hold all shear force data for this point load

    for i, x in enumerate(X):
        shear = 0  #Initialise the shear force for this data point
        moment = 0  #Initialise the bending moment for this data point

        if x>A:        # calculate the shear and moment due to reaction at A
          shear = shear + Va
          moment = moment - Va*(x-A)

        if x>B:  # calculate the shear and moment due to reaction at B
          shear = shear + Vb
          moment = moment - Vb*(x-B)

        if x>xm:   # calculate the moment influence of point moment ( No impact on the shear ) 
          moment = moment - m
            
         # Store shear and moment for this location       
        Shear[i] = shear
        Moment[i] = moment

    return Shear, Moment



#Define function to calculate shear forces and bending moments due to UDLs

In [ ]:
def shear_moment_UDL(n):
    xStart = distributedLoads[n,0] 
    xEnd = distributedLoads[n,1]
    fy = distributedLoads[n,2]
    Va = UDL_record[n,0]  # Vertical reaction at A for this UDL
    Vb = UDL_record[n,1]  # Vertical reaction at B for this UDL

    # Cycle through the structure and calculate the shear force and bending moment at each point
    Shear = np.zeros(len(X))  #Initialise a container to hold all shear force data for this point load
    Moment = np.zeros(len(X)) #Initialise a container to hold all shear force data for this point load

    for i, x in enumerate(X):
        shear = 0  #Initialise the shear force for this data point
        moment = 0  #Initialise the bending moment for this data point

        if x>A:        # calculate the shear and moment due to reaction at A
          shear = shear + Va
          moment = moment - Va*(x-A)

        if x>B:  # calculate the shear and moment due to reaction at B
          shear = shear + Vb
          moment = moment - Vb*(x-B)

        if x>xStart and x<=xEnd :   # calculate the shear and moment due to point load
          shear = shear + fy*(x-xStart)
          moment = moment - fy*(x-xStart)*0.5*(x-xStart)
        elif (x>xEnd):
            shear = shear + fy*(xEnd-xStart)
            moment = moment - fy*(xEnd-xStart)*(x-xStart-0.5*(xEnd - xStart))
            
         # Store shear and moment for this location       
        Shear[i] = shear
        Moment[i] = moment

    return Shear, Moment



#Define function to calculate shear forces and bending moments due to LDLs

In [ ]:
def shear_moment_LDL(n):
    xStart = linearLoads[n,0]
    xEnd = linearLoads[n,1]
    fy_start = linearLoads[n,2]
    fy_end = linearLoads[n,3]
    Va = LDL_record[n,0]  # Vertical reaction at A for this LDL
    Vb = LDL_record[n,1]  # Vertical reaction at B for this LDL

    # Cycle through the structure and calculate the shear force and bending moment at each point
    Shear = np.zeros(len(X))  #Initialise a container to hold all shear force data for this point load
    Moment = np.zeros(len(X)) #Initialise a container to hold all shear force data for this point load

    for i, x in enumerate(X):
        shear = 0  #Initialise the shear force for this data point
        moment = 0  #Initialise the bending moment for this data point


        if x>A:        # calculate the shear and moment due to reaction at A
          shear = shear + Va
          moment = moment - Va*(x-A)

        if x>B:  # calculate the shear and moment due to reaction at B
          shear = shear + Vb
          moment = moment - Vb*(x-B)

        if x>xStart and x<xEnd:
            if abs(fy_start)>0:
                x_base = x-xStart # base of the triangular load distribution 
                f_cut = fy_start - x_base*(fy_start/(xEnd-xStart)) # Mag of LDL at cut
                R1 = 0.5*x_base*(fy_start-f_cut)  # Mag of resultant for triangular portion of load
                R2 = x_base*f_cut # Mag of resultant for constant portion of load
                shear = shear +R1 +R2
                moment = moment - R1*(2/3)*x_base - R2*(x_base/2)
                
            else:
                x_base = x-xStart
                f_cut = fy_end *(x_base/(xEnd-xStart))
                R = 0.5*x_base*f_cut
                shear = shear + R
                moment = moment -R*(x_base/3)
                
        elif x>xEnd:
            if abs(fy_start)<0:
                R = 0.5*fy_star * (xEnd-xStart)
                xr = xStart + (1/3)*(xEnd - xStart)
                shear = shear + R
                moment = moment - R*(x-xr)
                
            else:
                R = 0.5*fy_end*(xEnd - xStart)
                xr = xStart + (2/3)*(xEnd - xStart)
                shear = shear + R
                moment = moment - R*(x-xr)
         # Store shear and moment for this location       
        Shear[i] = shear
        Moment[i] = moment

    return Shear, Moment    

    

Cycle through all point loads and determine shear and moment

In [ ]:
if(nPL>0):
      for n, p in enumerate(pointLoads):
        Shear, Moment =  shear_moment_PL(n)
        shearForce = np.append(shearForce, [Shear], axis = 0) # Store shear force for each point load
        bendingMoment = np.append(bendingMoment, [Moment], axis = 0) # Store bending moment for each point load



Cycle through all point moments and determine shear and moment

In [ ]:
if(nPM>0):
    for n, p in enumerate(pointMoments):
        Shear, Moment = shear_moment_PM(n)
        shearForce = np.append(shearForce, [Shear], axis = 0) # Store shear force for each point moment
        bendingMoment = np.append(bendingMoment, [Moment], axis = 0) # Store bending moment for each point moment
        

Cycle through all UDLs and determine shear and moment

In [ ]:
if(nUDL>0):
    for n, p in enumerate(distributedLoads):
        Shear, Moment = shear_moment_UDL(n)
        shearForce = np.append(shearForce, [Shear], axis = 0) # Store shear force for each UDL
        bendingMoment = np.append(bendingMoment, [Moment], axis = 0) # Store bending moment for each UDL
        

Cycle through all LDLs and determine shear and moment

In [ ]:
if(nLDL>0):
    for n, p in enumerate(linearLoads):
        Shear, Moment = shear_moment_LDL(n)
        shearForce = np.append(shearForce, [Shear], axis = 0) # Store shear force for each LDL
        bendingMoment = np.append(bendingMoment, [Moment], axis = 0) # Store bending moment for each LDL


Printing and Plotting

In [ ]:
print('The vertical reaction at A (Va) is',reactions[0], 'kN')
print('The horizontal reaction at A (Ha) is',reactions[1], 'kN')
print('The vertical reaction at B (Vb) is',reactions[2], 'kN')

In [ ]:
#print('The vertical reaction at A (Va) is {one} kN'. format(one=round(reactions[0],2))) # for rounding

Plotly

In [ ]:
!pip install plotly
import plotly as py
import plotly.graph_objs as gob

py.offline.init_notebook_mode(connected=True) # setup offline plotting

In [ ]:
# Define layout object
layout = gob.Layout(title={'text': 'Shear Force Diagram', 'y':0.80, 'x':0.5, 'xanchor':'center', 'yanchor':'top'},
yaxis = dict(title ='Shear Force (kN)'),
xaxis = dict(title = 'Distance (m)', range = [-1, span+1]),
showlegend = False,)

line = gob.Scatter(x = X,
                  y = sum(shearForce),
                  mode = 'lines',
                  name = 'Shear Force',
                    fill = 'tonexty',
                  line_color = 'blue',
                  fillcolor = 'rgba(0,0,255,0.3)')

axis = gob.Scatter(
    x =[0, span],
    y = [0,0],
    mode = 'lines',
    line_color = 'black'
) 

fig = gob.Figure(data =[line,axis], layout=layout)

py.offline.iplot(fig)


In [ ]:
# Define layout object
layout = gob.Layout(title={'text': 'Bending Moment Diagram', 'y':0.80, 'x':0.5, 'xanchor':'center', 'yanchor':'top'},
yaxis = dict(title ='Bending Moment (kNm)', autorange = 'reversed'),
xaxis = dict(title = 'Distance (m)', range = [-1, span+1]),
showlegend = False,)

line = gob.Scatter(x = X,
                  y = -sum(bendingMoment),
                  mode = 'lines',
                  name = 'Bending Moment',
                    fill = 'tonexty',
                  line_color = 'green',
                  fillcolor = 'rgba(0,255,0,0.3)')

axis = gob.Scatter(x =[0, span],
    y = [0,0],
    mode = 'lines',
    line_color = 'black'
) 

fig = gob.Figure(data =[line,axis], layout=layout)

py.offline.iplot(fig)
